In [7]:
import tweepy
from positive_tweet import positive_tweet
import HTMLParser
import json

In [9]:
with open('twitter_creds2.json', 'w') as f:
    json.dump(credentials, f, ensure_ascii=False)

In [28]:
# Import API Keys
with open('twitter_creds.json') as creds:    
    tw_creds = json.load(creds)

In [29]:
#Access the twitter API
consumer_key = tw_creds['consumer_key']
consumer_secret = tw_creds['consumer_secret']
access_token = tw_creds['access_token']
access_token_secret = tw_creds['access_secret']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [30]:
#Set user to follow and pull tweets from
username = 'realDonaldTrump'
public_tweets = api.user_timeline(username)
user = api.get_user(username)
print "Now listening to account: " + username

Now listening to account: realDonaldTrump


In [31]:
#post positive version of tweet
def post_tweet(original_tweet):
    n=0
    posttweet = positive_tweet(original_tweet.text)
    #loop through and see if it comes up with a shorter tweet if over 140 characters
    while n <100 and len(posttweet)>140:
        n=n+1
        posttweet = positive_tweet(original_tweet.text)
    #If the original tweet is the same as the modified one (no ngative words found), retweet the original.  Otherwise, post the modified tweet.
    if HTMLParser.HTMLParser().unescape(original_tweet.text) != posttweet:
        try:
            api.update_status(posttweet)
            print "Tweet posted"
            print original_tweet.text
            print posttweet
        except:
            print "too long, not posted"
    else:
        api.retweet(original_tweet.id)

In [ ]:
#Stream posts from twitter and post positive tweets
class MyStreamListener(tweepy.StreamListener):

    def on_status(self, status):
        #when an event is detected print the text of the event and then post the positive tweet
        print("New tweet detected")
        print status.text
        #post_tweet(status)


myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)
myStream.filter(track=[str(user.id)])
